In [1]:
## 模型与剖分

import math
import numpy as np
from numpy.linalg import solve
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt

from fealpy.decorator import cartesian
from fealpy.mesh import TriangleMesh
from fealpy.functionspace import LagrangeFiniteElementSpace
from fealpy.boundarycondition import DirichletBC

from tool import getIsBdNode, uniform_refine, get_cr_node_cell, H_1_norm
from tool import get_cr_glam_and_pre, get_phi_val,get_phi_grad_and_div, get_A1_A2_F, my_solve
from tool import get_stiff_and_div_matrix

n   = 4  #剖分次数
n   = n + 1 
Lam = [1,1e1,1e2,1e3,1e4,1e5]
Mu  = Lam

H     = np.zeros(n)                              #步长
P     = np.zeros(len(Lam))                       #误差阶
E     = np.zeros((len(Lam),n), dtype=np.float64) #每个lambda(行)对应的误差(列)

class PDE():
    def __init__(self, mu=1, lam=1):
        self.mu  = mu
        self.lam = lam
        
    def domain(self):
        return [0, 1, 0, 1]
    
    def init_mesh(self, n=1, meshtype='tri'):
        node = np.array([
            (0,0),
            (1,0),
            (1,1),
            (0,1)], dtype=np.float64)
        cell = np.array([(1,2,0), (3,0,2)], dtype=np.int64)
        mesh = TriangleMesh(node, cell)
        mesh.uniform_refine(n)
        return mesh
    
    @cartesian
    def source(self, p):
        x   = p[..., 0]
        y   = p[..., 1]
        mu  = self.mu
        lam = self.lam
        
        sin = np.sin
        cos = np.cos
        val = np.zeros(p.shape, dtype=np.float64)
        
        val[..., 0] = -((2 * mu + lam) * y * (y - 1) * (2 * cos(x) - (x - 1) * sin(x))
                        + (mu + lam) * (2 * x - 1) * (sin(y) + (y - 1) * cos(y)) 
                        + 2 * mu * (x -1) * sin(x))
        val[..., 1] = -((2 * mu + lam) * x * (x - 1) * (2 * cos(y) - (y - 1) * sin(y))
                        + (mu + lam) * (2 * y - 1) * (sin(x) + (x - 1) * cos(x))
                        + 2 * mu * (y - 1) * sin(y))

        #val[..., 0] = -(2 * (2 * mu + lam) * y * (y - 1) 
        #                + (mu + lam) * (2 * x - 1) * (2 * y - 1)
        #                + 2 * mu * x * (x - 1))
        #val[..., 1] = -(2 * (2 * mu + lam) * x * (x - 1)
        #                + (mu + lam) * (2 * x - 1) * (2 * y - 1)
        #                + 2 * mu * y * (y - 1))
        return val
    
    def dirichlet(self, p):
        var = np.zeros_like(p)
        return var
    
    def solution(self, p):
        x = p[..., 0]
        y = p[..., 1]
        
        val = np.zeros(p.shape, dtype=np.float64)
        
        val[..., 0] = y * (x - 1) * (y - 1) * np.sin(x)
        val[..., 1] = x * (x - 1) * (y - 1) * np.sin(y)
        
        #val[..., 0] = x * (x - 1) * y * (y - 1)
        #val[..., 1] = x * (x - 1) * y * (y - 1)
        return val
        
    def is_dirichlet_boundary(self, p):
        x = p[..., 0]
        y = p[..., 1]
        flag = np.max(np.abs(y)) < 1e-13 
        return flag
    
def error(u, uh):
    e = u - uh
    emax = np.max(np.abs(e))
    return emax

def print_error(Lam, H, E):
    for i in range(len(Lam)):
        print("---------------------Lam= {}---------------------".format(Lam[i]))
        #print("lam= ", Lam[i])
        print()
        for j in range(n):
            print("h= ", H[j])
            print("e=", E[i][j])
            #print("e_rel= ", E_rel[i,j])
            print()
        print()
    
def print_P(Lam, P):
    print("---------------------误差阶---------------------")
    for i in range(len(Lam)):
        print("lam= ", Lam[i])
        print("p= ", P[i])
        print()
              
for i in range(len(Lam)):
    pde = PDE(Mu[i], 1)# / 2 + 10, Lam[i])
    node = np.array([
            (0,0),
            (1,0),
            (1,1),
            (0,1)], dtype=np.float64)
    cell = np.array([(1,2,0), (3,0,2)], dtype=np.int64)
    for j in range(n):
        NC = cell.shape[0]
        #print("cr_NC= ", NC)
        # nn 特定情况下剖分次数
        nn  = math.log(NC/2, 4)
        NN = int(3 * 2 * 4**nn - (3 * 2 * 4**nn - 4 * 2**nn) / 2)
        cm = np.ones(NC, dtype=np.float64) / NC
     
        cr_node, cr_cell = get_cr_node_cell(node,cell)
        #print("cr_node= ", cr_node.shape[0])
        #print("cr_cell= ", cr_cell.shape[0])
        
        # cr_node_cell [NC,3,2]
        cr_node_cell = cr_node[cr_cell]
        #print("cr_node_cell= ", cr_node_cell)
        
        cr_glam_x_y, cr_glam_x_y_pre = get_cr_glam_and_pre(cr_node, cr_cell)
            
        A1, A2 = get_stiff_and_div_matrix(cr_node, cr_cell, cm)
        
        # phi_val [NC,3(点),6(6个基函数),2(两个分量)]
        phi_node_val = get_phi_val(node, cell, cr_glam_x_y_pre)
        
        # val [NC,3(点),2(分量)] 右端项在各顶点的值
        val = pde.source(node[cell])
        #print("val= ", val[0,0])
        
        # phi_val [NC,3,6] 基函数和右端项的点乘
        phi_val = np.einsum("cijk, cik -> cij", phi_node_val, val)
        #print("phi_val= ", phi_val[0,0])
        # bb [NC,6]
        bb = phi_val.sum(axis=1) * cm[0] / 3
        #print("bb= ", bb.shape)
        #print("cm= ", cm[0])

        #F = np.zeros(2 * NN)
        #np.add.at(F, cell_x_y, bb)
        ##print("F= ", F)
        
        A1, A2, F = get_A1_A2_F(A1, A2, bb, cr_node, cr_cell)
        A = pde.mu * A1 + (pde.lam + pde.mu) * A2
              
        uh = my_solve(A, F, cr_node, getIsBdNode)
        u = pde.solution(cr_node)
        H[j] = np.sqrt(2 * cm[0])
        #print("h= ", np.sqrt(2 * cm[0]))
        # 计算误差
        E[i][j] = error(u, uh)
        #E[i][j] = H_1_norm(u-uh, H[j])
        #print("u= ", u)
        #print("uh= ", uh)
        node, cell = uniform_refine(node, cell)

if n-1 > 1: 
    for i in range(len(Lam)):
        fig = plt.figure()
        plt.plot(np.log(H[1:]), np.log(E[i][1:]))
        plt.title("lam={}".format(Lam[i]))
        plt.xlabel("log(h)")
        plt.ylabel("log(e)")
        plt.savefig(fname="CRFem/elasticityCRFemLam_{}.png"
                    .format(Lam[i]))
        plt.close(fig)
        
    for i in range(len(Lam)):
        f = np.polyfit(np.log(H[1:]), np.log(E[i][1:]) ,1)
        P[i] = f[0]

#print("u= ", np.max(u))
#print("uh= ", np.max(uh))
print_error(Lam, H, E)
#print_error(Lam, H, E[isInterNode], E_rel[isInterNode])
if n-1 > 1:
    print_P(Lam, P)

---------------------Lam= 1---------------------

h=  1.0
e= 0.29011206542651247

h=  0.5
e= 0.07987830915813865

h=  0.25
e= 0.020326196583183388

h=  0.125
e= 0.0058054494674396225

h=  0.0625
e= 0.0015808927711673066


---------------------Lam= 10.0---------------------

h=  1.0
e= 0.18652932253106827

h=  0.5
e= 0.049859206858231926

h=  0.25
e= 0.012611314864767902

h=  0.125
e= 0.003404342414751668

h=  0.0625
e= 0.0009062659789250459


---------------------Lam= 100.0---------------------

h=  1.0
e= 0.17617104824152385

h=  0.5
e= 0.04690166490822197

h=  0.25
e= 0.01186494113300688

h=  0.125
e= 0.0031691945046611585

h=  0.0625
e= 0.0008399728017311715


---------------------Lam= 1000.0---------------------

h=  1.0
e= 0.1751352208125694

h=  0.5
e= 0.04660654318840584

h=  0.25
e= 0.01179065378369782

h=  0.125
e= 0.003145757453778021

h=  0.0625
e= 0.0008333619340446449


---------------------Lam= 10000.0---------------------

h=  1.0
e= 0.17503163806967395

h=  0.5
e= 0.046